# Preprocessing, Filters

The presented material is based on material prepared by *Jonas Hartmann (Gilmour group, EMBL Heidelberg)*

## Table of Contents

1. [Background](#Background)
1. [Convolutional Filters](#Convolutional-Filters)
1. [Rank Filters]()

## Background

The goal of image preprocessing is to prepare or optimize the images to make further analysis easier. Usually, this boils down to increasing the signal-to-noise ratio by removing noise and background and by enhancing structures of interest.

The specific preprocessing steps used in a pipeline depend on the type of sample, the microscopy technique used, the image quality, and the desired downstream analysis. 

The most common operations include:


- Deconvolution
    - Image reconstruction based on information about the PSF of the microscope
    - These days deconvolution is often included with microscope software
    - *Our example images are not deconvolved, but will do just fine regardless*


- Conversion to 8-bit images to save memory / computational time
    - *Our example images are already 8-bit*


- Cropping of images to an interesting region
    - *The field of view in our example images is fine as it is*


- Smoothing of technical noise
    - This is a very common step and usually helps to improve almost any type of downstream analysis
    - Commonly used filters are the `Gaussian filter` and the `median filter`
    - *Here we will be using a Gaussian filter.*


- Corrections of technical artifacts
    - Common examples are uneven illumination and multi-channel bleed-through
    - *Here we will deal with uneven signal by adaptive/local thresholding*


- Background subtraction
    - There are various ways of sutracting background signal from an image
    - Two different types are commonly distinguished:
        - `uniform background subtraction` treats all regions of the image the same
        - `adaptive or local background subtraction` automatically accounts for differences between regions of the image
    - *Here we will do something similar to adaptive background subtraction when we do adaptive thresholding*

## Prerequisite: Load an image

In [ ]:
%matplotlib inline
import imageio
import io
import numpy as np
import requests
import skimage

from matplotlib import pyplot as plt
from matplotlib import cm

In [ ]:
#load and display an image
image_file = (
    "https://git.embl.de/grp-bio-it/image-analysis-training-resources/raw/"
    "1fc71747f0caddfd3e206b700b41f2bf69c07a49/image_data/xy_8bit__two_cells.tif"
)

buffer = io.BytesIO()
buffer.write(requests.get(image_file).content)
buffer.seek(0)

image = imageio.imread(buffer, format='tif')
f = plt.figure()
s = f.add_subplot(111)
s.imshow(image, cmap=cm.gray)

## Convolutional Filters

### Gaussian Smoothing

A Gaussian filter smoothens an image by convolving it with a Gaussian-shaped kernel. In the case of a 2D image, the Gaussian kernel is also 2D and will look something like this:

<img src="../session-3to5/ipynb_images/gaussian_kernel_grid.png" alt="Gaussian Kernel Figure" style="width: 300px;"/>

How much the image is smoothed by a Gaussian kernel is determined by the standard deviation  of the Gaussian distribution, usually referred to as **sigma** ($\sigma$). A higher $\sigma$ means a broader distribution and thus more smoothing.

**How to choose the correct value of $\sigma$?**

This depends a lot on your images, in particular on the pixel size. In general, the chosen $\sigma$ should be large enough to blur out noise but small enough so the "structures of interest" do not get blurred too much. Usually, the best value for $\sigma$ is simply found by trying out some different options and looking at the result.


Useful links:

* [scikit-image filters](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian)


In [ ]:
sigma = 3
filtered_image = skimage.filters.gaussian(image, sigma=sigma)
f = plt.figure()
s = f.add_subplot(121)
s.imshow(image, cmap=cm.gray)
s.set_title("original image")
s = f.add_subplot(122)
s.imshow(filtered_image, cmap=cm.gray)
s.set_title(f"filtered image with $\sigma$ = {sigma}")